# Konnektor - Building Ligand Networks

this is Konnektor, which you can use to construct different types of Networks.

In [ ]:
# NBVAL_SKIP
# Only run this cell if on google colab
import os
if "COLAB_RELEASE_TAG" in os.environ:
    !pip install -U https://github.com/conda-incubator/condacolab/archive/cuda-version-12.tar.gz
    import condacolab
    condacolab.install_mambaforge()
    !wget -q https://raw.githubusercontent.com/OpenFreeEnergy/konnektor/main/examples/environment.yaml
    !mamba env update -q --name=base --file=environment.yaml
    
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
import numpy as np
from rdkit import Chem

try:
    from kartograf import KartografAtomMapper, MappingRMSDScorer
except ModuleNotFoundError:
    !{sys.executable} -m pip install git+https://github.com/OpenFreeEnergy/kartograf.git
    from kartograf import KartografAtomMapper, MappingRMSDScorer


import logging
from kartograf.atom_mapper import logger
logger.setLevel(logging.ERROR)


In [ ]:
from kartograf import SmallMoleculeComponent
from konnektor.visualization import draw_ligand_network, draw_network_widget

mapper = KartografAtomMapper()
scorer = MappingRMSDScorer()

from konnektor.data import get_benzene_ligands
compounds = get_benzene_ligands()
compounds = list(filter(lambda x: not x.name in ["lig_2", "lig_3", "lig_4", "lig_7"], compounds))
Chem.Draw.MolsToGridImage ([c.to_rdkit() for c in compounds], )

# Radial Network Layout

In [ ]:
from konnektor.network_planners import StarNetworkGenerator
ligand_network_planner = StarNetworkGenerator(mapper=mapper, scorer=scorer, 
                                       n_processes=1)

In [ ]:
radial_network = ligand_network_planner(compounds)
radial_network.name="Star Network"
radial_network

In [ ]:
# NBVAL_SKIP
draw_network_widget(radial_network);

## Cyclic Network Layout

In [ ]:
from konnektor.network_planners import CyclicNetworkGenerator

ligand_network_planner = CyclicNetworkGenerator(mapper=mapper, 
                                                       scorer=scorer)

In [ ]:
cyclic_network = ligand_network_planner(compounds)
cyclic_network.name="Cyclic Network"
cyclic_network

In [ ]:
# NBVAL_SKIP
draw_network_widget(cyclic_network);

## Starry Sky Network Layout

In [ ]:
from konnektor.network_planners import StarrySkyNetworkGenerator

ligand_network_planner = StarrySkyNetworkGenerator(mapper=mapper, 
                                                       scorer=scorer)

In [ ]:
starry_sky_network = ligand_network_planner(compounds)
starry_sky_network.name="Starry Sky Network"
starry_sky_network

In [ ]:
# NBVAL_SKIP
draw_network_widget(starry_sky_network);